# TD Règles d’association 

Ce répertoire contient les fichiers et solutions pour le TD Règles d'association.  
**Fait par : CHICA Miller et TORRES Jonathan**


## 1. Implémentation de trois méthodes de *Frequent Itemset Generation*

### 1.

### 2.

### 3.

---

## 2. Comparaison des implémentations de Section 1

### 1.

### 2.

---

## 3. Comparaison avec les implémentations de *MLxtend* et *Spark MLlib*

### 1.

### 2.

### 3.

---